<a href="https://colab.research.google.com/github/cyyeh/time-series-playground/blob/main/pytorch-101/pytorch_lightning_101.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# PyTorch Lightning 101

Adapt the [PyTorch quickstart example](https://github.com/cyyeh/time-series-playground/blob/main/pytorch-101/quickstart.ipynb) to PyTorch Lightning!

In [1]:
#@title Install Required Dependencies { vertical-output: true }

!pip install pytorch_lightning

     |████████████████████████████████| 808kB 29.3MB/s 
     |████████████████████████████████| 645kB 49.6MB/s 
     |████████████████████████████████| 112kB 53.0MB/s 
     |████████████████████████████████| 276kB 49.3MB/s 
     |████████████████████████████████| 829kB 37.9MB/s 
     |████████████████████████████████| 10.6MB 45.1MB/s 
     |████████████████████████████████| 1.3MB 48.7MB/s 
     |████████████████████████████████| 296kB 52.9MB/s 
     |████████████████████████████████| 143kB 56.1MB/s 
  Created wheel for future: filename=future-0.18.2-cp37-none-any.whl size=491070 sha256=7123ff3a7de824d9c054291f23e8c7efb64565e9b5179f27b5610084a87e1ef1
  Stored in directory: /root/.cache/pip/wheels/8b/99/a0/81daf51dcd359a9377b110a8a886b3895921802d2fc1b2397e
Successfully built future
ERROR: tensorflow 2.5.0 has requirement tensorboard~=2.5, but you'll have tensorboard 2.4.1 which is incompatible.
  Found existing installation: PyYAML 3.13
    Uninstalling PyYAML-3.13:
      Successfully un

In [2]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision.datasets import FashionMNIST
from torchvision.transforms import ToTensor
import pytorch_lightning as pl

## Working with data

In [3]:
# Download training data from open datasets.
training_data = FashionMNIST(
    root='data',
    train=True,
    download=True,
    transform=ToTensor(),
)

# Download test data from open datasets.
test_data = FashionMNIST(
    root='data',
    train=False,
    download=True,
    transform=ToTensor(),
)


Extracting data/FashionMNIST/raw/train-images-idx3-ubyte.gz to data/FashionMNIST/raw




Extracting data/FashionMNIST/raw/train-labels-idx1-ubyte.gz to data/FashionMNIST/raw




Extracting data/FashionMNIST/raw/t10k-images-idx3-ubyte.gz to data/FashionMNIST/raw




Extracting data/FashionMNIST/raw/t10k-labels-idx1-ubyte.gz to data/FashionMNIST/raw

Processing...
Done!


/usr/local/lib/python3.7/dist-packages/torchvision/datasets/mnist.py:502: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  /pytorch/torch/csrc/utils/tensor_numpy.cpp:143.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


In [4]:
batch_size = 64

# Create data loaders.
train_dataloader = DataLoader(training_data, batch_size=batch_size)
test_dataloader = DataLoader(test_data, batch_size=batch_size)

for X, y in test_dataloader:
  print(f'Shape of X[N, C, H, W]: {X.shape}')
  print(f'Shape of y: {y.shape} {y.dtype}')
  break

Shape of X[N, C, H, W]: torch.Size([64, 1, 28, 28])
Shape of y: torch.Size([64]) torch.int64


## Creating Models

In [19]:
class NeuralNetwork(pl.LightningModule):
  def __init__(self):
    super().__init__()
    self.flatten = nn.Flatten()
    self.linear_relu_stack = nn.Sequential(
        nn.Linear(28*28, 512),
        nn.ReLU(),
        nn.Linear(512, 512),
        nn.ReLU(),
        nn.Linear(512, 10),
        nn.ReLU()
    )

  def forward(self, x):
    x = self.flatten(x)
    logits = self.linear_relu_stack(x)
    return logits

  def configure_optimizers(self):
    return torch.optim.SGD(self.parameters(), lr=1e-3)
  
  def training_step(self, train_batch, batch_idx):
    X, y = train_batch
    pred = self(X)
    loss_fn = nn.CrossEntropyLoss()
    loss = loss_fn(pred, y)
    return loss

  def validation_step(self, val_batch, batch_idx):
    X, y = val_batch
    pred = self(X)
    loss_fn = nn.CrossEntropyLoss()
    loss = loss_fn(pred, y)

In [20]:
model = NeuralNetwork()

trainer = pl.Trainer(gpus=1, max_epochs=5, check_val_every_n_epoch=1)
trainer.fit(model, train_dataloader, test_dataloader)

GPU available: True, used: True
TPU available: False, using: 0 TPU cores
LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]

  | Name              | Type       | Params
-------------------------------------------------
0 | flatten           | Flatten    | 0     
1 | linear_relu_stack | Sequential | 669 K 
-------------------------------------------------
669 K     Trainable params
0         Non-trainable params
669 K     Total params
2.679     Total estimated model params size (MB)
